In [26]:
import os
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow import keras
import numpy as np
train = pd.read_csv('acc_train_shuff.csv') 

In [27]:
train_data = train.drop("Activity", axis=1) # drop labels for training set
train_data = train_data.drop("subject", axis=1)
train_target= train["Activity"].copy()

In [28]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_train=scaler.fit_transform(train_data)

In [29]:
from sklearn.preprocessing import LabelEncoder
encoder=LabelEncoder()
y_train=encoder.fit_transform(train_target)
y_train=pd.get_dummies(y_train).values

In [33]:
def build_model():
    # create model
    model = keras.models.Sequential()

    model.add(keras.layers.Dense(64, input_dim=X_train.shape[1] , activation='relu'))
    model.add(keras.layers.Dense(64, activation='relu'))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.Dense(128, activation='relu'))
    model.add(keras.layers.Dense(64, activation='relu'))
    model.add(keras.layers.Dense(32, activation='relu'))
    model.add(keras.layers.Dense(6, activation='softmax'))

    model.compile(optimizer = keras.optimizers.Adam(lr = 0.0005),loss='categorical_crossentropy', metrics=['accuracy'])

    return model

In [34]:
k = 10
num_val = len(X_train) // k
num_epochs = 22
all_scores = []
for i in range(k):
    print('processing fold #', i)
    # Prepare the validation data: data from partition # k
    X_val = X_train[i * num_val: (i + 1) * num_val]
    Y_val = y_train[i * num_val: (i + 1) * num_val]

    # Prepare the training data: data from all other partitions
    X_train_part = np.concatenate(
        [X_train[:i * num_val],
         X_train[(i + 1) * num_val:]],
        axis=0)
    Y_train_part = np.concatenate(
        [y_train[:i * num_val],
         y_train[(i + 1) * num_val:]],
        axis=0)

    # Build the Keras model (already compiled)
    model = build_model()
    # Train the model (in silent mode, verbose=0)
    model.fit(X_train_part, Y_train_part,epochs=num_epochs,  batch_size=256 , verbose=0)
    # Evaluate the model on the validation data 
    val_loss, val_acc = model.evaluate(X_val, Y_val, verbose=0)
    all_scores.append(val_acc)

processing fold # 0
processing fold # 1
processing fold # 2
processing fold # 3
processing fold # 4
processing fold # 5
processing fold # 6
processing fold # 7
processing fold # 8
processing fold # 9


In [35]:
np.mean(all_scores)

0.9685715